In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/pruning_quantization", "")))

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import MLP_Models, LSTM_Models, GRU_Models
from utils.benchmark import Benchmark
from compact.pruning import PruneModel

In [2]:
conf = Config()
load_model = LSTM_Models()
model_conf = load_model.lstm_4
model = load_model.get_model(model_conf)
pruning = PruneModel()
print(conf.device)

cuda


In [3]:
# data loader
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, model_type=load_model.type)

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

dataset = NetworkDataset(X_test, y_test)
loader = DataLoader(dataset, conf.batch_size, shuffle=True)

In [4]:
# prune model
name = model_conf["name"]
result_path = os.path.join(conf.benchmark_host, "pruned_quantized_model", "pruned_" + name + ".txt")
model.load()

# new hidden sizes
if "mlp" in name:
    # prune linear layers
    pruned_model, keep = pruning.prune_linear_units(model.model)

    # set new hidden sizes
    pruned_model.hidden_sizes = [layer.out_features for (layer, _, _) in pruned_model.linear]

    # prune BatchNorm
    pruned_model.bn = pruning.prune_batchnorm(pruned_model.bn, keep)

else:
    pruned_model, keep = pruning.prune_rnn_units(model.model)
    pruned_model.bn1 = pruning.prune_batchnorm(pruned_model.bn1, keep)
    pruned_model, keep = pruning.prune_linear_units(pruned_model, in_features=keep)
    pruned_model.linear_sizes = [layer.out_features for (layer, _, _) in pruned_model.linear]
    pruned_model.bn2 = pruning.prune_batchnorm(pruned_model.bn2, keep)

model.model = pruned_model.to(conf.device)
model.optimizer = torch.optim.AdamW(
    model.model.parameters(), 
    lr=conf.learning_rate, 
    weight_decay=conf.weight_decay
)
model.scheduler = torch.optim.lr_scheduler.ExponentialLR(
    model.optimizer, 
    gamma=conf.gamma
)
model.checkpoint_path = model_conf["pruned_checkpoint_path"]

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/large_model/lstm_4.pth!


In [5]:
# fine-tune pruned model
f1_macro, f1_micro, roc_auc_macro = model.train(train_loader, val_loader, epochs=1)

Epoch: 1/1, Macro-F1 score: 0.70, Micro-F1 score: 0.95, Macro ROC AUC score: 0.96, Train loss: 0.204, Val loss: 0.072


In [6]:
# test model
loss, metrics = model.evaluate(loader)
print(f"Macro-F1 score: {metrics['f1_macro']:.2f}, Micro-F1 score: {metrics['f1_micro']:.2f}, Macro ROC AUC score: {metrics['roc_auc_macro']:.2f}")

Macro-F1 score: 0.71, Micro-F1 score: 0.95, Macro ROC AUC score: 0.98


In [7]:
#model.save()